In [ ]:
#|default_exp utils
#|default_cls_lvl 3

In [ ]:
#|hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# utils

> Various utility classes and functions used by the `BLURR` library.

In [ ]:
#|export
import importlib, sys, torch
from typing import Any, Callable, Dict, List, Optional, Union, Type

from fastcore.all import *
from fastai.callback.all import *
from fastai.imports import *
from fastai.learner import *
from fastai.losses import BaseLoss, BCEWithLogitsLossFlat, CrossEntropyLossFlat, MSELossFlat
from fastai.torch_core import *
from fastai.torch_imports import *


In [ ]:
#| echo: false
import pdb

from IPython.display import display
from fastcore.test import *
from nbdev.showdoc import show_doc


In [ ]:
#|hide
#|cuda
torch.cuda.set_device(1)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #1: GeForce GTX 1080 Ti


## Utility classes

In [ ]:
#|export
class Singleton:
    def __init__(self, cls):
        self._cls, self._instance = cls, None

    def __call__(self, *args, **kwargs):
        if self._instance == None:
            self._instance = self._cls(*args, **kwargs)
        return self._instance


`Singleton` functions as python decorator.  Use this above any class to turn that class into a singleton (see [here](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html) for more info on the singleton pattern).

In [ ]:
@Singleton
class TestSingleton:
    pass


a = TestSingleton()
b = TestSingleton()
test_eq(a, b)


## Utility methods

In [ ]:
#|export
def str_to_type(typename: str) -> type:  # The name of a type as a string  # Returns the actual type
    "Converts a type represented as a string to the actual class"
    return getattr(sys.modules[__name__], typename)


In [ ]:
show_doc(str_to_type, title_level=3)


<h3 id="str_to_type" class="doc_header"><code>str_to_type</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>str_to_type</code>(**`typename`**:`str`)

Converts a type represented as a string to the actual class

||Type|Default|Details|
|---|---|---|---|
|**`typename`**|`str`||*No Content*|


How to use:

In [ ]:
print(str_to_type("test_eq"))
print(str_to_type("TestSingleton"))


<function test_eq at 0x7f01efe0a700>


In [ ]:
#|export
def print_versions(
    # A string of space delimited package names or a list of package names
    packages: Union[str, List[str]]
):
    """Prints the name and version of one or more packages in your environment"""
    packages = packages.split(" ") if isinstance(packages, str) else packages

    for item in packages:
        item = item.strip()
        print(f"{item}: {importlib.import_module(item).__version__}")


In [ ]:
show_doc(print_versions, title_level=3)


<h3 id="print_versions" class="doc_header"><code>print_versions</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>print_versions</code>(**`packages`**:`Union`\[`str`, `typing.List[str]`\])

Prints the name and version of one or more packages in your environment

||Type|Default|Details|
|---|---|---|---|
|**`packages`**|`typing.Union[str, typing.List[str]]`||A string of space delimited package names or a list of package names|


How to use:

In [ ]:
print_versions("torch transformers fastai")
print("---")
print_versions(["torch", "transformers", "fastai"])


torch: 1.10.1+cu111
transformers: 4.18.0
fastai: 2.6.3
---
torch: 1.10.1+cu111
transformers: 4.18.0
fastai: 2.6.3


In [ ]:
#|export
# see the following threads for more info:
# - https://forums.fast.ai/t/solved-reproducibility-where-is-the-randomness-coming-in/31628?u=wgpubs
# - https://docs.fast.ai/dev/test.html#getting-reproducible-results
def set_seed(seed_value: int = 42):
    """This needs to be ran before creating your DataLoaders, before creating your Learner, and before each call
    to your fit function to help ensure reproducibility.
    """
    np.random.seed(seed_value)  # cpu vars
    torch.manual_seed(seed_value)  # cpu vars
    random.seed(seed_value)  # python

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # gpu vars
        torch.backends.cudnn.deterministic = True  # needed
        torch.backends.cudnn.benchmark = False


In [ ]:
show_doc(set_seed, title_level=3)

<h3 id="set_seed" class="doc_header"><code>set_seed</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h3>

> <code>set_seed</code>(**`seed_value`**:`int`=*`42`*)

This needs to be ran before creating your DataLoaders, before creating your Learner, and before each call
to your fit function to help ensure reproducibility.

## Loss functions

In [ ]:
#|export
class PreCalculatedLoss(BaseLoss):
    """
    If you want to let your Hugging Face model calculate the loss for you, make sure you include the `labels` argument in your inputs and use 
    `PreCalculatedLoss` as your loss function. Even though we don't really need a loss function per se, we have to provide a custom loss class/function 
    for fastai to function properly (e.g. one with a `decodes` and `activation` methods).  Why?  Because these methods will get called in methods 
    like `show_results` to get the actual predictions.
    
    Note: The Hugging Face models ***will always*** calculate the loss for you ***if*** you pass a `labels` dictionary along with your other inputs 
    (so only include it if that is what you intend to happen)
    """
    def __call__(self, inp, targ, **kwargs):
        return tensor(0.0)


class PreCalculatedCrossEntropyLoss(PreCalculatedLoss, CrossEntropyLossFlat):
    pass


class PreCalculatedBCELoss(PreCalculatedLoss, BCEWithLogitsLossFlat):
    pass


class PreCalculatedMSELoss(PreCalculatedLoss):
    def __init__(self, *args, axis=-1, floatify=True, **kwargs):
        super().__init__(nn.MSELoss, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)


In [ ]:
#|export
class MultiTargetLoss(Module):
    """
    Provides the ability to apply different loss functions to multi-modal targets/predictions.
    
    This new loss function can be used in many other multi-modal architectures, with any mix of loss functions.  
    For example, this can be ammended to include the `is_impossible` task, as well as the start/end token tasks 
    in the SQUAD v2 dataset (or in any extractive question/answering task)
    """

    def __init__(
        self,
        # The loss function for each target
        loss_classes: List[Callable] = [CrossEntropyLossFlat, CrossEntropyLossFlat],
        # Any kwargs you want to pass to the loss functions above
        loss_classes_kwargs: List[dict] = [{}, {}],
        # The weights you want to apply to each loss (default: [1,1])
        weights: Union[List[float], List[int]] = [1, 1],
        # The `reduction` parameter of the lass function (default: 'mean')
        reduction: str = "mean",
    ):
        loss_funcs = [cls(reduction=reduction, **kwargs) for cls, kwargs in zip(loss_classes, loss_classes_kwargs)]
        store_attr(self=self, names="loss_funcs, weights")
        self._reduction = reduction

    # custom loss function must have either a reduction attribute or a reduction argument (like all fastai and
    # PyTorch loss functions) so that the framework can change this as needed (e.g., when doing lear.get_preds
    # it will set = 'none'). see this forum topic for more info: https://bit.ly/3br2Syz
    @property
    def reduction(self):
        return self._reduction

    @reduction.setter
    def reduction(self, v):
        self._reduction = v
        for lf in self.loss_funcs:
            lf.reduction = v

    def forward(self, outputs, *targets):
        loss = 0.0
        for i, loss_func, weights, output, target in zip(range(len(outputs)), self.loss_funcs, self.weights, outputs, targets):
            loss += weights * loss_func(output, target)

        return loss

    def activation(self, outs):
        acts = [self.loss_funcs[i].activation(o) for i, o in enumerate(outs)]
        return acts

    def decodes(self, outs):
        decodes = [self.loss_funcs[i].decodes(o) for i, o in enumerate(outs)]
        return decodes


## Export -

In [ ]:
#|hide
from nbdev import nbdev_export

nbdev_export()


Converted 00_callbacks.ipynb.
Converted 00_utils.ipynb.
Converted 01_text-callbacks.ipynb.
Converted 01_text-utils.ipynb.
Converted 11_text-data-core.ipynb.
Converted 11_text-modeling-core.ipynb.
Converted 12_text-data-language-modeling.ipynb.
Converted 12_text-modeling-language-modeling.ipynb.
Converted 13_text-data-token-classification.ipynb.
Converted 13_text-modeling-token-classification.ipynb.
Converted 14_text-data-question-answering.ipynb.
Converted 14_text-modeling-question-answering.ipynb.
Converted 20_text-data-seq2seq-core.ipynb.
Converted 20_text-modeling-seq2seq-core.ipynb.
Converted 21_text-data-seq2seq-summarization.ipynb.
Converted 21_text-modeling-seq2seq-summarization.ipynb.
Converted 22_text-data-seq2seq-translation.ipynb.
Converted 22_text-modeling-seq2seq-translation.ipynb.
Converted 99a_text-examples-high-level-api.ipynb.
Converted 99b_text-examples-glue.ipynb.
Converted 99c_text-examples-glue-plain-pytorch.ipynb.
Converted 99d_text-examples-multilabel.ipynb.
Conv